# Part 0: Dataloader and Visualizations

In [1]:
import numpy as np
from PIL import Image
import scipy.io
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import wandb

from voc_dataset import VOCDataset
from utils import *

USE_WANDB = False
%reload_ext autoreload 

## Q0.1: Editing the Dataloader
The first part of the assignment involves editing the dataloader so that we can access bounding-box proposals as well as the ground-truth bounding boxes. The ground truth bounding box can be accessed through the VOC Dataset annotations itself and we have completed this part for you in the starter code. 

Unsupervised bounding box proposals are obtained through methods such as [Selective Search](https://ivi.fnwi.uva.nl/isis/publications/2013/UijlingsIJCV2013/UijlingsIJCV2013.pdf). Since Selective Search is slow to run on each image, we have pre-computed the bounding box proposals for you (you downloaded this in the data preparation step).

Your task is to change the dataloader to obtain the proposed bounding boxes for each image. Feel free to experiment with the data in the files to figure out the number of proposals per image, their scores, etc. Returning a dictionary would be convenient here. For the bounding boxes, using the relative positions is usually a better idea since they are invariant to changes in the size of the image.

In [2]:
dataset = VOCDataset('trainval', top_n=10)

In [8]:
# for key, value in dataset.roi_data:
# boxes = sorted(enumerate(dataset.roi_data['boxScores'][0][0]), key= lambda x: x[1], reverse = True)
# for k, v in boxes:
#     print (k, v)
# print("``````````````````````````````````````````````````````````````````````````````````````````````````")
# for k, v in enumerate(dataset.roi_data['boxScores'][0][0]):
#     print (k, v)
# print(type(dataset.roi_data['boxes'][0][0]))
# s = dataset.__getitem__(2020)
# print(s["rois"])

# boxes = dataset.roi_data['boxes'][0][2020]
# print(type(boxes[0]))
# boxScores = dataset.roi_data['boxScores'][0][2020][:, 0]
# print(boxScores.shape)
# boxScores_sorted = np.argsort(-boxScores)
# boxScores_sorted = boxScores_sorted[:dataset.top_n]
# # b = boxScores[boxScores_sorted]
# # print(b)
# final_boxes = boxes[boxScores_sorted]
# print(final_boxes.shape)
# for box in final_boxes:
#     print(type(box))

# final_boxes = []
# for num in range(len(boxScores_sorted)):
#     print(boxes[num,:])
#     final_boxes.append(boxes[num,:])

# print(final_boxes)
# print(dataset[0])
import matplotlib.pyplot as plt
idx = 1000
data = dataset[idx]
# plt.show(data['image'].numpy())
print(data['image'].shape)
print(data['label'].shape)
print(data["gt_classes"])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

**Q0.1**: Load the image corresponding to index 2020 and print the GT labels associated with it.

**Hint**: items at a particular index can be accesed by usual indexing notation (dataset[idx])

In [3]:
# TODO: get the image information from index 2020
idx = 2020
data = dataset[idx]
print(data['label'])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0.])


## Q0.2 and Q0.3: Wandb Logging
First, let's initialize a Weights and Biases project. 

In [4]:
USE_WANDB = True
if USE_WANDB:
    wandb.init(project="vlr-hw1", reinit=True)

wandb: Currently logged in as: parth_malpathak (use `wandb login --relogin` to force relogin)


**Q0.2**: Complete this block for overlaying the ground truth box on an image.

**Hint**: convert the image tensor to a PIL image and plot it (check `utils.py` for helper functions). You can use [this](https://docs.wandb.ai/library/log) as a reference for logging syntax.

In [5]:
class_id_to_label = dict(enumerate(dataset.CLASS_NAMES))

# TODO: load the GT information corresponding to index 2020.
original_image = tensor_to_PIL(data["image"])
gt_labels = data["gt_classes"]
gt_boxes = data["gt_boxes"]
img = wandb.Image(original_image, boxes={
    "predictions": {
        "box_data": get_box_data(gt_labels, gt_boxes),
        "class_labels": class_id_to_label,
    },
})
# TODO: log the GT bounding box
# PIL_image = tensor_to_PIL(original_image)
wandb.log({"inputs": img})

**Q0.3**: Visualize the top 10 bounding proposals corresponding to index 2020.

**Hint**: Check the `get_box_data` function in `utils.py` and understand how it is being used. 

In [6]:
# TODO: plot top ten proposals (of bounding boxes)
proposals = data["rois"]
proposals_for_image = proposals[:, [1, 0, 3, 2]]
# print(proposal_for_image)
classes = np.arange(proposals.shape[0]).tolist()
proposal_image = wandb.Image(original_image, boxes={
"predictions": {
    "box_data": get_box_data(classes, proposals_for_image),
},
})
wandb.log({"inputs": proposal_image})